# Ohnologs & eQTLs

### Background

Enrichment for eQTLs of ohnologs and nonohnologs.

### Imports

In [1]:
import pandas as pd
import re
import os.path

### Functions

In [2]:
def removeGeneIDVersions(text):
    return re.findall('(ENSG\d+)', text)[0]

### Input/Output files

---

### eQTL affected tissues per gene
[Back to top](#Outline)

In [3]:
genesWitheQTLTissueCountBonferroni = pd.read_csv('../../outputFiles/cGTExGenesWithCounts.csv')
genesWitheQTLTissueCountBonferroni.head()

,expressedTissues,Ensembl Gene ID,affectedTissues,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand
0,23,ENSBTAG00000008414,2.0,20,41213852,41297142,1
1,23,ENSBTAG00000006129,2.0,25,25087968,25327424,1
2,23,ENSBTAG00000006086,3.0,19,14587761,14613432,1
3,23,ENSBTAG00000055051,2.0,24,25141833,25187671,-1
4,23,ENSBTAG00000032433,1.0,17,52750181,52762092,-1


##### Ohnologs

In [4]:
ohnologsRelaxed = pd.read_csv('../../datasets/geneLists/Singh and Isambert/btaurus.Pairs.Relaxed.2R.Ens96.1-X.txt')
ohnologsRelaxed.head()

,Ensembl Gene ID
0,ENSBTAG00000032079
1,ENSBTAG00000021457
2,ENSBTAG00000003097
3,ENSBTAG00000016838
4,ENSBTAG00000016794


In [5]:
SSDsRelaxedOhnos = pd.read_csv('../../datasets/geneLists/SSDsRelaxedOhnos.cow.txt')
SSDsRelaxedOhnos.head()

,Ensembl Gene ID
0,ENSBTAG00000006648
1,ENSBTAG00000054829
2,ENSBTAG00000046015
3,ENSBTAG00000001150
4,ENSBTAG00000026259


In [6]:
singletons = pd.read_csv('../../datasets/geneLists/singletons.cow.txt')
singletons.head()

,Ensembl Gene ID
0,ENSBTAG00000021251
1,ENSBTAG00000051226
2,ENSBTAG00000013841
3,ENSBTAG00000011528
4,ENSBTAG00000012594


In [7]:
genesWitheQTLTissueCountBonferroni.loc[:, 'type'] = ''
genesWitheQTLTissueCountBonferroni.loc[genesWitheQTLTissueCountBonferroni['Ensembl Gene ID'].isin(ohnologsRelaxed['Ensembl Gene ID']) , 'type'] = 'ohno'
genesWitheQTLTissueCountBonferroni.loc[genesWitheQTLTissueCountBonferroni['Ensembl Gene ID'].isin(SSDsRelaxedOhnos['Ensembl Gene ID']) , 'type'] = 'SSD'
genesWitheQTLTissueCountBonferroni.loc[genesWitheQTLTissueCountBonferroni['Ensembl Gene ID'].isin(singletons['Ensembl Gene ID']) , 'type'] = 'singleton'
genesWitheQTLTissueCountBonferroni.groupby('type').describe()

Chromosome/scaffold name                                             \
                             count       mean       std  min  25%   50%   75%   
type                                                                            
SSD                         9685.0  13.299535  8.044332  1.0  6.0  13.0  19.0   
ohno                        5947.0  12.736842  8.095390  1.0  5.0  11.0  19.0   
singleton                   3791.0  13.382485  8.036680  1.0  6.0  13.0  19.0   

                Gene end (bp)                ...  affectedTissues        \
            max         count          mean  ...              75%   max   
type                                         ...                          
SSD        29.0        9685.0  4.977519e+07  ...              2.0  20.0   
ohno       29.0        5947.0  4.965163e+07  ...              2.0  17.0   
singleton  29.0        3791.0  4.927084e+07  ...              2.0  16.0   

          expressedTissues                                                    
                     count       mean       std  min   25%   50%   75%   max  
type                                                                          
SSD                 9685.0  16.918018  7.956803  1.0  11.0  22.0  23.0  23.0  
ohno                5947.0  19.112830  6.271700  1.0  17.0  23.0  23.0  23.0  
singleton           3791.0  19.385650  6.701911  1.0  19.0  23.0  23.0  23.0  

[3 rows x 48 columns]

In [8]:
genesWitheQTLTissueCountBonferroni.head()

,expressedTissues,Ensembl Gene ID,affectedTissues,Chromosome/scaffold name,Gene start (bp),Gene end (bp),Strand,type
0,23,ENSBTAG00000008414,2.0,20,41213852,41297142,1,ohno
1,23,ENSBTAG00000006129,2.0,25,25087968,25327424,1,singleton
2,23,ENSBTAG00000006086,3.0,19,14587761,14613432,1,SSD
3,23,ENSBTAG00000055051,2.0,24,25141833,25187671,-1,SSD
4,23,ENSBTAG00000032433,1.0,17,52750181,52762092,-1,SSD


In [9]:
genesWitheQTLTissueCountBonferroni['propOfExpressedAffectedByeQTL'] = genesWitheQTLTissueCountBonferroni['affectedTissues'] / genesWitheQTLTissueCountBonferroni['expressedTissues']

In [10]:
genesWitheQTLTissueCountBonferroni.groupby('type').describe()

Chromosome/scaffold name                                             \
                             count       mean       std  min  25%   50%   75%   
type                                                                            
SSD                         9685.0  13.299535  8.044332  1.0  6.0  13.0  19.0   
ohno                        5947.0  12.736842  8.095390  1.0  5.0  11.0  19.0   
singleton                   3791.0  13.382485  8.036680  1.0  6.0  13.0  19.0   

                Gene end (bp)               ...  expressedTissues        \
            max         count          mean ...               75%   max   
type                                        ...                           
SSD        29.0        9685.0  4.977519e+07 ...              23.0  23.0   
ohno       29.0        5947.0  4.965163e+07 ...              23.0  23.0   
singleton  29.0        3791.0  4.927084e+07 ...              23.0  23.0   

          propOfExpressedAffectedByeQTL                                \
                                  count      mean       std  min  25%   
type                                                                    
SSD                              9685.0  0.088787  0.136118  0.0  0.0   
ohno                             5947.0  0.070009  0.090550  0.0  0.0   
singleton                        3791.0  0.079539  0.113927  0.0  0.0   

                                    
                50%       75%  max  
type                                
SSD        0.043478  0.130435  1.0  
ohno       0.043478  0.090909  1.0  
singleton  0.043478  0.095238  1.0  

[3 rows x 56 columns]

In [11]:
genesWitheQTLTissueCountBonferroni[genesWitheQTLTissueCountBonferroni['affectedTissues']>0].groupby('type').describe()

Chromosome/scaffold name                                             \
                             count       mean       std  min  25%   50%   75%   
type                                                                            
SSD                         5779.0  13.469631  8.054627  1.0  6.0  13.0  19.0   
ohno                        3821.0  12.836169  8.070816  1.0  6.0  11.0  19.0   
singleton                   2431.0  13.629782  8.002232  1.0  7.0  13.0  19.0   

                Gene end (bp)               ...  expressedTissues        \
            max         count          mean ...               75%   max   
type                                        ...                           
SSD        29.0        5779.0  5.170979e+07 ...              23.0  23.0   
ohno       29.0        3821.0  5.114440e+07 ...              23.0  23.0   
singleton  29.0        2431.0  5.016714e+07 ...              23.0  23.0   

          propOfExpressedAffectedByeQTL                                \
                                  count      mean       std       min   
type                                                                    
SSD                              5779.0  0.148798  0.148735  0.043478   
ohno                             3821.0  0.108962  0.092288  0.043478   
singleton                        2431.0  0.124036  0.121334  0.043478   

                                              
                25%       50%       75%  max  
type                                          
SSD        0.047619  0.090909  0.173913  1.0  
ohno       0.043478  0.086957  0.130435  1.0  
singleton  0.043478  0.086957  0.136364  1.0  

[3 rows x 56 columns]

In [12]:
genesWitheQTLTissueCountBonferroni.to_csv("../../outputFiles/genesWitheQTLTissueCountBonferroniAndOhnologStatus.cow.csv", sep='\t')